# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
# Split Cabin into Deck, CabinNumber, and Side
spaceship[['Deck', 'CabinNumber', 'Side']] = spaceship['Cabin'].str.split('/', expand=True)

# Convert CabinNumber to numeric
spaceship['CabinNumber'] = pd.to_numeric(spaceship['CabinNumber'], errors='coerce')

# Drop the columns that we don't need
spaceship.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)

In [4]:
# Classify the non-numeric columns
non_numeric_cols = spaceship[['HomePlanet', 'Destination', 'Deck', 'Side']]

# Do dummy encoding on the non-numeric columns
spaceship = pd.get_dummies(spaceship, columns=non_numeric_cols.columns, drop_first=True)

In [5]:
spaceship.corrwith(spaceship['Transported']).sort_values(ascending=False)

Transported                  1.000000
CryoSleep                    0.468645
HomePlanet_Europa            0.176916
Deck_B                       0.144733
Deck_C                       0.108193
Side_S                       0.101455
FoodCourt                    0.046566
HomePlanet_Mars              0.019544
Deck_G                       0.016269
ShoppingMall                 0.010141
Destination_PSO J318.5-22    0.000092
Deck_T                      -0.014568
Deck_D                      -0.034046
VIP                         -0.037650
CabinNumber                 -0.045097
Age                         -0.075026
Deck_F                      -0.087753
Destination_TRAPPIST-1e     -0.094700
Deck_E                      -0.097965
VRDeck                      -0.207075
Spa                         -0.221131
RoomService                 -0.244611
dtype: float64

In [6]:
spaceship.dropna(inplace=True)

**Perform Train Test Split**

In [7]:
#Select the columns you want to use for your model
X = spaceship[['CryoSleep', 'HomePlanet_Europa', 'RoomService', 'Side_S', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Deck_B', 'Destination_TRAPPIST-1e', 'Age']]
y = spaceship['Transported'] 

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Normalize and standarize data**

In [8]:
# Normalize data
from sklearn.preprocessing import MinMaxScaler, StandardScaler

normalizer = MinMaxScaler()

normalizer.fit(X_train)

MinMaxScaler()

In [9]:
X_train_norm = normalizer.fit_transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [10]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_train_norm.head()

,CryoSleep,HomePlanet_Europa,RoomService,Side_S,FoodCourt,ShoppingMall,Spa,VRDeck,Deck_B,Destination_TRAPPIST-1e,Age
0,0.0,0.0,0.10504,1.0,0.000000,0.001532,0.000000,0.000000,0.0,1.0,0.189873
1,0.0,0.0,0.00000,0.0,0.103311,0.000298,0.000000,0.003295,0.0,0.0,0.253165
2,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.024902,0.000049,0.0,1.0,0.291139
3,0.0,1.0,0.00000,0.0,0.026599,0.000000,0.003883,0.077055,0.0,1.0,0.683544
4,1.0,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.088608


In [11]:
# Standardize data
standardizer = StandardScaler()

standardizer.fit(X_train)

StandardScaler()

In [12]:
X_train_std = standardizer.fit_transform(X_train)

X_test_std = standardizer.transform(X_test)

In [13]:
X_test_std = pd.DataFrame(X_test_std, columns = X_test.columns)
X_test_std.head()

,CryoSleep,HomePlanet_Europa,RoomService,Side_S,FoodCourt,ShoppingMall,Spa,VRDeck,Deck_B,Destination_TRAPPIST-1e,Age
0,-0.728272,-0.570626,-0.348335,-1.006550,-0.288483,0.203240,-0.273823,0.102883,-0.32487,-1.459181,0.764297
1,1.373112,-0.570626,-0.348335,0.993492,-0.288483,-0.277229,-0.274702,-0.267862,-0.32487,0.685316,0.833307
2,-0.728272,-0.570626,-0.235114,-1.006550,-0.285387,-0.277229,0.025806,-0.102304,-0.32487,0.685316,-0.684912
3,1.373112,-0.570626,-0.348335,-1.006550,-0.288483,-0.277229,-0.274702,-0.267862,-0.32487,0.685316,-0.270852
4,-0.728272,-0.570626,-0.321602,-1.006550,-0.081715,0.820102,-0.259765,0.054448,-0.32487,0.685316,0.695287


In [14]:

X_test_std = X_test_std.values
X_train_norm = X_train_norm.values


**Model testing**

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression(random_state=42)

log_reg.fit(X_train_std, y_train)

y_pred_logreg = log_reg.predict(X_test_std)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_logreg))

Logistic Regression Accuracy: 0.799576570218772


In [16]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=8)

knn.fit(X_train_std, y_train)

y_pred_knn = knn.predict(X_test_std)
print("KNN Accuracy:", accuracy_score(y_test, y_pred_knn))

KNN Accuracy: 0.7678193366266761


In [17]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=10, random_state=42)

tree.fit(X_train_std, y_train)

y_pred_tree = tree.predict(X_test_std)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_tree))

Decision Tree Accuracy: 0.7854622441778405


In [18]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X_train_norm, y_train)

y_pred_rf = rf.predict(X_test_norm)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

Random Forest Accuracy: 0.803105151729005


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

In [19]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

- Bagging and Pasting

In [20]:
base_clf = DecisionTreeClassifier(random_state=42)

bagging_clf = BaggingClassifier(n_estimators=100, bootstrap=True, random_state=42)
bagging_clf.fit(X_train_std, y_train)
y_pred_bagging = bagging_clf.predict(X_test_std)
print("Bagging accuracy:", accuracy_score(y_test, y_pred_bagging))

Bagging accuracy: 0.7960479887085392


In [21]:
pasting_clf = BaggingClassifier(n_estimators=100, bootstrap=False, random_state=42)
pasting_clf.fit(X_train_std, y_train)
y_pred_pasting = pasting_clf.predict(X_test_std)
print("Pasting accuracy:", accuracy_score(y_test, y_pred_pasting))

Pasting accuracy: 0.7501764290755116


- Random Forests

In [22]:
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_clf.fit(X_train_std, y_train)
y_pred_rf_ens = random_forest_clf.predict(X_test_std)
print("Random Forest accuracy:", accuracy_score(y_test, y_pred_rf_ens))

Random Forest accuracy: 0.8009880028228652


- Gradient Boosting

In [23]:
gradient_boosting_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gradient_boosting_clf.fit(X_train_std, y_train)
y_pred_gb = gradient_boosting_clf.predict(X_test_std)
print("Gradient Boosting accuracy:", accuracy_score(y_test, y_pred_gb))

Gradient Boosting accuracy: 0.8045165843330981


- Adaptive Boosting

In [24]:
ada_boost_clf = AdaBoostClassifier(n_estimators=100, learning_rate=0.5, random_state=42)
ada_boost_clf.fit(X_train_std, y_train)
y_pred_ada = ada_boost_clf.predict(X_test_std)
print("AdaBoost accuracy:", accuracy_score(y_test, y_pred_ada))

c:\Users\almuo\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost accuracy: 0.8038108680310515


Which model is the best and why?

In [25]:
# The best model is Random Forest, because it has a high accuracy and takes less time.